In [1]:
%cd ..

/hdd/aouadt/these/projets/3d_segm


In [19]:
from importlib import reload
import random
from functools import reduce
from os.path import join
from time import time
import os
import re

from tqdm import tqdm
import pandas as pd
import cv2
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch.utils.data.dataloader import DataLoader
import pytorch_lightning as pl
import skimage.morphology as morp
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import TensorBoardLogger
from PIL import Image, ImageDraw
import torchviz
from deep_morpho.datasets.mnist_dataset import MnistClassifDataset
from deep_morpho.models import LightningBiMoNNClassifier, BiSE, COBiSE, BiSEC, COBiSEC


import deep_morpho.models.softplus as sfp
import general.array_morphology as arm
import general.structuring_elements as selm
from general.nn.loss import DiceLoss
import deep_morpho.threshold_fn as threshold_fn
import deep_morpho.morp_operations as morpop
import deep_morpho.models.threshold_layer as threshold_layer
import deep_morpho.models.complementation_layer as complementation_layer
import deep_morpho.models.pconv as pconv
import deep_morpho.models.lightning_pconv as lpconv
import deep_morpho.models.bise as bise
import deep_morpho.models.lui as lui
import deep_morpho.models.bisel as bisel
import deep_morpho.models.cobise as cobise
import deep_morpho.models.dilation_sum_layer as dis
import deep_morpho.models.bimonn as bimonn
import deep_morpho.models as models
import deep_morpho.models.lightning_bise as lbise
import deep_morpho.models.lightning_cobise as lcobise
import deep_morpho.models.lightning_bimonn as lbimonn
import deep_morpho.datasets.generate_forms1 as gfo
import deep_morpho.datasets.generate_forms2 as gfo2
import deep_morpho.datasets.generate_forms3 as gfo3
import deep_morpho.datasets.axspa_roi_dataset as axd
import deep_morpho.datasets.multi_rect_dataset as mrda
import deep_morpho.datasets.mnist_dataset as mnist_dataset
import deep_morpho.observables.plot_parameters as obs_weights
import deep_morpho.observables.plot_pred as obs_pred
import deep_morpho.observables.weight_histogram as weight_histogram
import deep_morpho.observables as obs
import general.nn.viz.plot_histogram as phist
import general.nn.viz.element_image as eltimage
import general.nn.viz.element_histogram as elthistogram
import deep_morpho.viz.morp_operations_viz as mov
import deep_morpho.viz.elt_generator_bimonn_forward_save as eltgenbifor
import deep_morpho.viz.elt_generator_bimonn_histogram as eltgenbihist
import deep_morpho.viz.bimonn_viz as bimonn_viz
import deep_morpho.save_results_template.load_args as largs
import deep_morpho.save_results_template.display_results as dr

def reload_modules():
    for modl in [sfp, arm, selm, threshold_fn, morpop,
                 threshold_layer,complementation_layer, pconv, lpconv, bise, lui, bisel, cobise,
                 dis, bimonn, models, lbise, lbimonn, lcobise, gfo, gfo2, gfo3, axd,
                 mrda, mnist_dataset, obs_weights, obs_pred, obs,
                weight_histogram, phist, eltimage, elthistogram, mov, eltgenbifor, eltgenbihist, bimonn_viz, largs, dr]:
        reload(modl)
        
reload_modules()

device = 'cpu'
if torch.cuda.is_available():
    device = 'cuda'
print(device)

cuda


# Individual Experiments

In [ ]:
reload_modules()
# tb_path = ("deep_morpho/results/DGMM_2022/sandbox/1/axspa_roi/version_7")
# tb_path = "/hdd/aouadt/these/projets/3d_segm/deep_morpho/results/results_tensorboards/Bimonn_exp_56/sandbox/2/softplus/diskorect/opening/disk/version_1"
# tb_path = "deep_morpho/results/results_tensorboards/Bimonn_exp_59/sandbox/0/softplus/diskorect/translation_erosion/hstick/version_0"
tb_path = "deep_morpho/results/results_tensorboards/Bimonn_exp_59/sandbox/0/softplus/diskorect/translation_erosion/dcross/version_0"

tb_path = join(tb_path, 'checkpoints', os.listdir(join(tb_path, 'checkpoints'))[0])

model = models.LightningBiMoNN.load_from_checkpoint(tb_path)
model.to(device)

loss = nn.BCELoss()

# Multi Experiments comparison

In [3]:
def list_dir_joined(folder: str):
    return [os.path.join(folder, k) for k in os.listdir(folder)]

path_global = "deep_morpho/results/results_tensorboards/Bimonn_mega_multi_1/softplus/diskorect/"
all_paths = []
for operation in os.listdir(path_global):
    if os.path.isdir(join(path_global, operation)):
        for selem in os.listdir(join(path_global, operation)):
            all_paths += sorted(list_dir_joined(join(path_global, operation, selem)), key=lambda x: int(
                re.findall(r'version_(\d+)$', x)[0]
            ))[:-1]

In [20]:
reload_modules()
results = dr.DisplayResults().get_all_results_from_tensorboard(all_paths, load_long_args=False)

Reading results: 100%|██████████| 272/272 [00:02<00:00, 91.69it/s] 


In [21]:
for r in results:
    r.update(r['args'])

In [28]:
df = pd.DataFrame(results)
df['dice'] = df['dice'].astype(float)
df['binary_mode_dice'] = df['binary_mode_dice'].astype(float)

In [13]:
df.columns

Index(['args', 'tb_path', 'weights', 'normalized_weights', 'bias', 'dice',
       'baseline_dice', 'convergence_dice', 'activation_P', 'learned_selem',
       'convergence_layer', 'target_selem', 'target_operation',
       'learned_weights_viz', 'learned_selems_viz', 'binary_mode_dice',
       'stopping_reason', 'optimizer', 'operations', 'loss_data',
       'experiment_name', 'experiment_subname', 'name', 'dataset_type',
       'dataset_path', 'n_inputs', 'learning_rate', 'batch_size',
       'num_workers', 'freq_imgs', 'n_epochs', 'n_atoms', 'atomic_element',
       'kernel_size', 'init_weight_mode', 'constant_activation_P',
       'constant_weight_P', 'threshold_mode', 'alpha_init', 'share_weights',
       'loss', 'patience_loss', 'patience_reduce_lr', 'closest_selem_method',
       'closest_selem_distance_fn', 'bias_optim_mode', 'loss_data_str'],
      dtype='object')

In [34]:
changing = ['loss_data_str', 'init_weight_mode', 'learning_rate', 'bias_optim_mode']
metrics = ['dice', 'binary_mode_dice']
df.loc[:, changing + metrics]

,loss_data_str,init_weight_mode,learning_rate,bias_optim_mode,dice,binary_mode_dice
0,MaskedBCELoss,KAIMING_UNIFORM,0.01,POSITIVE,8.130714e-01,0.441327
1,MaskedMSELoss,KAIMING_UNIFORM,0.01,POSITIVE,8.149265e-01,0.420488
2,MaskedDiceLoss,KAIMING_UNIFORM,0.01,POSITIVE,9.983954e-01,1.000000
3,MaskedBCELoss,KAIMING_UNIFORM,0.1,POSITIVE,2.593382e-09,0.524210
4,MaskedMSELoss,KAIMING_UNIFORM,0.1,POSITIVE,9.985447e-01,1.000000
...,...,...,...,...,...,...
267,MaskedDiceLoss,CUSTOM_HEURISTIC,0.1,POSITIVE,1.000000e+00,1.000000
268,MaskedBCELoss,CUSTOM_CONSTANT,0.01,POSITIVE,9.805872e-01,0.895800
269,MaskedMSELoss,CUSTOM_CONSTANT,0.01,POSITIVE,9.923737e-01,0.990035
270,MaskedDiceLoss,CUSTOM_CONSTANT,0.01,POSITIVE,9.820556e-01,0.905236


In [35]:
df.groupby(changing)[metrics].mean()

dice  \
loss_data_str  init_weight_mode learning_rate bias_optim_mode             
MaskedBCELoss  CUSTOM_CONSTANT  0.01          POSITIVE         0.806939   
                                0.1           POSITIVE         0.998906   
               CUSTOM_HEURISTIC 0.01          POSITIVE         0.549370   
                                0.1           POSITIVE         0.549024   
               KAIMING_UNIFORM  0.01          POSITIVE         0.665337   
                                0.1           POSITIVE         0.434480   
MaskedDiceLoss CUSTOM_CONSTANT  0.01          POSITIVE         0.996631   
                                0.1           POSITIVE         0.990897   
               CUSTOM_HEURISTIC 0.01          POSITIVE         0.811241   
                                0.1           POSITIVE         0.729951   
               KAIMING_UNIFORM  0.01          POSITIVE         0.777878   
                                0.1           POSITIVE         0.615549   
MaskedMSELoss  CUSTOM_CONSTANT  0.01          POSITIVE         0.927271   
                                0.1           POSITIVE         0.833333   
               CUSTOM_HEURISTIC 0.01          POSITIVE         0.659495   
                                0.1           POSITIVE         0.555556   
               KAIMING_UNIFORM  0.01          POSITIVE         0.626560   
                                0.1           POSITIVE         0.481572   

                                                               binary_mode_dice  
loss_data_str  init_weight_mode learning_rate bias_optim_mode                    
MaskedBCELoss  CUSTOM_CONSTANT  0.01          POSITIVE                 0.861460  
                                0.1           POSITIVE                 0.996120  
               CUSTOM_HEURISTIC 0.01          POSITIVE                 0.752841  
                                0.1           POSITIVE                 0.677616  
               KAIMING_UNIFORM  0.01          POSITIVE                 0.619558  
                                0.1           POSITIVE                 0.509117  
MaskedDiceLoss CUSTOM_CONSTANT  0.01          POSITIVE                 0.989471  
                                0.1           POSITIVE                 0.809359  
               CUSTOM_HEURISTIC 0.01          POSITIVE                 0.794002  
                                0.1           POSITIVE                 0.752737  
               KAIMING_UNIFORM  0.01          POSITIVE                 0.736560  
                                0.1           POSITIVE                 0.636348  
MaskedMSELoss  CUSTOM_CONSTANT  0.01          POSITIVE                 0.944595  
                                0.1           POSITIVE                 0.931714  
               CUSTOM_HEURISTIC 0.01          POSITIVE                 0.785066  
                                0.1           POSITIVE                 0.734992  
               KAIMING_UNIFORM  0.01          POSITIVE                 0.522428  
                                0.1           POSITIVE                 0.465326